In [3]:
# reload imports when code changes
%load_ext autoreload
QEMU_BIN = "../../qemu/build/qemu-x86_64"
QEMU_PATCHED_BIN = "./qemu/build/qemu-x86_64"

from assemble import assemble
from utils import exec_cmd
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
def get_qemu_cmd(ex):
    return QEMU_BIN + " -plugin ./inscount.so " + ex

def get_qemu_cmd_patched(ex):
    return QEMU_PATCHED_BIN + " " + ex

def bind_eff(cmd):
    eff_cores = exec_cmd("cat", ["/sys/devices/cpu_core/cpus"])
    if eff_cores.returncode != 0:
        print("Warning: I only have Intel computers so I don't know how to get the effective cores on your system")
        return cmd
    return "taskset -c " + eff_cores.stdout.strip() + " " + cmd


def run(cmd_func, nbNops = 100, nbRuns = 1000):
    ex = assemble("nop\n" * nbNops, "test")
    cmd = cmd_func(ex)
    cmd = "perf stat -j -e instructions,cycles,task-clock -r " + str(nbRuns) + " " + cmd
    cmd = bind_eff(cmd)
    print(cmd)
    cmd = cmd.split(" ")
    res = exec_cmd(cmd[0], cmd[1:])
    if res.returncode != 0:
        print("Error: " + res.stderr)
        return
    data = []
    for line in res.stderr.split("\n"):
        if len(line) == 0:
            continue
        data.append(json.loads(line))
    return (data, res.stdout, res.stderr)

taskset -c 0-11 perf stat -j -e instructions,cycles,task-clock -r 10 ./qemu/build/qemu-x86_64 test.out
([{'counter-value': '<not counted>', 'unit': '', 'event': 'cpu_atom/instructions:u/', 'variance': 0.0, 'event-runtime': 0, 'pcnt-running': 0.0, 'metric-value': '0.000000', 'metric-unit': ''}, {'counter-value': '8034791.000000', 'unit': '', 'event': 'cpu_core/instructions:u/', 'variance': 0.0, 'event-runtime': 2754102, 'pcnt-running': 100.0, 'metric-value': '0.000000', 'metric-unit': 'insn per cycle'}, {'counter-value': '<not counted>', 'unit': '', 'event': 'cpu_atom/cycles:u/', 'variance': 0.0, 'event-runtime': 0, 'pcnt-running': 0.0, 'metric-value': '0.000000', 'metric-unit': ''}, {'counter-value': '4737902.000000', 'unit': '', 'event': 'cpu_core/cycles:u/', 'variance': 1.55, 'event-runtime': 2754102, 'pcnt-running': 100.0, 'metric-value': '1.629958', 'metric-unit': 'GHz'}, {'counter-value': '2.906763', 'unit': 'msec', 'event': 'task-clock:u', 'variance': 2.17, 'event-runtime': 27541

In [11]:
def run_save(name, cmd_func, nbNops = 100, nbRuns = 1000):
    data, stdout, stderr = run(cmd_func, nbNops, nbRuns)
    with open(name + ".json", "w") as f:
        for d in data:
            f.write(json.dumps(d) + "\n")

In [14]:
for i in range(0, 10001, 1000):
    run_save("qemu_" + str(i), get_qemu_cmd, i, 5000)
    run_save("qemu_patched_" + str(i), get_qemu_cmd_patched, i, 5000)

taskset -c 0-11 perf stat -j -e instructions,cycles,task-clock -r 5000 ../../qemu/build/qemu-x86_64 -plugin ./inscount.so test.out
taskset -c 0-11 perf stat -j -e instructions,cycles,task-clock -r 5000 ./qemu/build/qemu-x86_64 test.out
